In [ ]:
import os
import sys
import scipy.io
import scipy.misc
import matplotlib.pyplot as plt
from matplotlib.pyplot import imshow
from PIL import Image
import numpy as np
import tensorflow as tf
import pprint
from cost_functions import *
from misc_utils import *
#from public_tests import *
%matplotlib inline

Load VGG19 

In [ ]:
img_size = 128
vgg = tf.keras.applications.VGG19(include_top=False,
                                  input_shape=(img_size, img_size, 3),
                                  weights='imagenet')

vgg = replace_maxpool_with_avgpool(vgg)
vgg.trainable = False

Chose your style and content layers. Uncomment the top to list all layers

In [ ]:
#Chose layers to use when calculating style cost

#for layer in vgg.layers:
#    print(layer.name)

STYLE_LAYERS = [
    ('block1_conv1', 0.2),#200
    #('block1_conv2', 0.2),
    ('block2_conv1', 0.2),#20
    #('block2_conv2', 0.2),
    ('block3_conv1', 0.2),#20
    #('block3_conv2', 0.2),
    #('block3_conv3', 0.2),
    ('block4_conv1', 0.2),#0.00002
    #('block4_conv2', 0.2),
    #('block4_conv3', 0.2),
    #('block4_conv4', 0.2),
    ('block5_conv1', 0.2),
    #('block5_conv2', 0.2),
    #('block5_conv3', 0.2),
    #('block5_conv4', 0.2)
    ]

CONTENT_LAYERS = [('block4_conv1', 1)]

INPUT_LAYERS = [(vgg.layers[0].name, 1)]

vgg_style_outputs = get_layer_outputs(vgg, STYLE_LAYERS)
vgg_model_content_outputs = get_layer_outputs(vgg, CONTENT_LAYERS)
inputs_layer_output = get_layer_outputs(vgg, INPUT_LAYERS)


In [ ]:
content_image, style_image, generated_image = load_images("images/sib.jpg", "images/monet.jpg", img_size=img_size, white_noise=True)

In [ ]:
# Assign the content image to be the input of the VGG model.  
# Set a_C to be the hidden layer activation from the layer we have selected
preprocessed_content =  tf.Variable(tf.image.convert_image_dtype(content_image, tf.float32))
a_C = vgg_model_content_outputs(preprocessed_content)
# Assign the input of the model to be the "style" image 
preprocessed_style =  tf.Variable(tf.image.convert_image_dtype(style_image, tf.float32))
a_S = vgg_style_outputs(preprocessed_style)

content_target = vgg_style_outputs(content_image)  # Content encoder
style_targets = vgg_style_outputs(style_image)     # Style encoder

In [ ]:
optimizer = tf.keras.optimizers.Adam(learning_rate=0.001)

#@tf.function()
def train_step(generated_image):
    with tf.GradientTape() as tape:
        # In this function you must use the precomputed encoded images a_S and a_C
        
        ### START CODE HERE
        
        # Compute a_G as the vgg_style_outputs for the current generated image
        #(1 line)
        a_G_s = vgg_style_outputs(generated_image)
        a_G_c = vgg_model_content_outputs(generated_image)
        
        # Getting the original image for denoising
        a_I = inputs_layer_output(generated_image)
        
        # Compute the style cost
        #(1 line)
        J_style = compute_style_cost(a_S, a_G_s, STYLE_LAYERS)
        
        #(2 lines)
        # Compute the content cost
        J_content = compute_content_cost(a_C,a_G_c)
        
        
        #J_noise = compute_noise_cost(a_I)
        J_noise = 0
        
        # Compute the total cost
        J = total_cost(J_content, J_style, J_noise, alpha = 4, beta = 4000, gamma = 0)
        
        ### END CODE HERE
        
    grad = tape.gradient(J, generated_image)

    optimizer.apply_gradients([(grad, generated_image)])
    generated_image.assign(clip_0_1(generated_image))
    
    # For grading purposes
    return J, J_style, J_content, J_noise

In [ ]:
# Show the generated image at some epochs
# Uncomment to reset the style transfer process. You will need to compile the train_step function again 
epochs = 1
for i in range(epochs):
    J, J_style, J_content, J_noise = train_step(generated_image)
    if i % 25 == 0:
        print(f"Epoch {i} with J: {J}, J_style: {J_style}, J_content: {J_content}, J_noise: {J_noise}")
        
    if i % 500 == 0:
        image = tensor_to_image(generated_image)
        imshow(image)
        image.save(f"output/image_test_{i}.jpg")
        plt.show() 

In [ ]:
a_G_s = vgg_style_outputs(generated_image)
J_style = compute_style_cost(a_S, a_G_s, STYLE_LAYERS, layer_wise_output=True)
for i, layer_cost in enumerate(J_style):
   print(f"Layer {STYLE_LAYERS[i]} cost: {layer_cost}")

print(tf.nn.softmax(J_style))

In [ ]:
# Show the 3 images in a row
fig = plt.figure(figsize=(16, 4))
ax = fig.add_subplot(1, 3, 1)
imshow(content_image[0])
ax.title.set_text('Content image')
ax = fig.add_subplot(1, 3, 2)
imshow(style_image[0])
ax.title.set_text('Style image')
ax = fig.add_subplot(1, 3, 3)
imshow(generated_image[0])
ax.title.set_text('Generated image')
plt.show()